In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import math
import os
from collections import defaultdict
from os import scandir

Generating FeatureSet!

In [ ]:
# accessing files from the directory and creating a SET a feature set by iterating through all the variables

dirpath = '/content/drive/My Drive/lingspam_public/lemm_stop/'

#Cleanerlist = (".","\'","\"","^",")","(","{","}",",","/","?",";","\"")

# Universal Feature Set creation by accessing all the files from the drive path
FeatureSet = set()

EmailMap = defaultdict(set)
EmailMapTF = defaultdict(list)

TotalFolders = 10

# Run through all the files in the folder
for pathnum in range(1, TotalFolders+1):
  Subfolder =dirpath + f"/part{pathnum}" 
  all_entries = scandir(Subfolder)

# For an entry in the all files 
  for entry in all_entries:
    Filepath = Subfolder + f"/{entry.name}"

# Readind a file 
    with open(Filepath, 'r') as thisfile:
      data = thisfile.read()
      # set for Bernoulli and Multinomial
      FileFeatureset = set(data.split())
      # List output for Term Frequency Calculations
      EmailMapTFfeatureMap = list(data.split())
      if Filepath not in EmailMap:
        EmailMap[str(Filepath)]= FileFeatureset

      if Filepath not in EmailMapTF:
        EmailMapTF[str(Filepath)] = EmailMapTFfeatureMap
        #print(EmailMap)
      FeatureSet = FeatureSet.union(FileFeatureset)      # Combing Feature Set

print(len(EmailMap))
print(FeatureSet)
#FeatureSet = FeatureSet.difference(set(Cleanerlist))
print(len(FeatureSet))
#print(EmailMap)



Generating 2 seperate DataFrame Objects for Term Frequency (TF) and Binary DataFrame Object

In [ ]:
import numpy as np
import csv 

# The FeatureSet is added 1 to Add SPAM Column
LastColumn = len(FeatureSet)
zeros_array = np.zeros( (len(EmailMap), LastColumn+1))
TF_arry = np.zeros( (len(EmailMap), LastColumn+1))
# Create a Map to zip in the index of the word in the FeatureSet
IndexMapofFeatureSet = {}
start = 0
for v in FeatureSet:
  if v not in IndexMapofFeatureSet:
    IndexMapofFeatureSet[v] = start
    start += 1

# Printing to check 
#print(IndexMapofFeatureSet)

# To fill the matrix of Emails x Words 
# Here 1 represents the presence of Words and 
# 0 represents absence
Row = 0
for K,V_Set in EmailMap.items():
  for Val in V_Set:
    zeros_array[Row][IndexMapofFeatureSet[str(Val)]] = 1
  if "spm" in K:
    zeros_array[Row][LastColumn] = 1
  Row += 1


# For Term Frequency 

Row = 0
for K,V_Set in EmailMapTF.items():
  for Val in V_Set:
    #if str(Val) not in Cleanerlist:
    TF_arry[Row][IndexMapofFeatureSet[str(Val)]] += 1
  if "spm" in K:
    TF_arry[Row][LastColumn] = 1
  Row += 1



In [ ]:
# List of FeatureSet
FeaturesList = list(FeatureSet)

FeaturesList.append('SPAM')

Adding DataFrame Objects to CSV File so that it is easier to Load next Time (Both Dataframes TF as well as Binary)

In [ ]:
FilepathTFDataSet = '/content/drive/My Drive/Consolidated_TF_EmailCSV.csv'

# writing to csv file  
with open(FilepathTFDataSet, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    csvwriter.writerow(FeaturesList)  
        
    # writing the data rows  
    csvwriter.writerows(TF_arry) 

In [ ]:
FilePath = '/content/drive/My Drive/ConsolidatedEmailCSV.csv'

# writing to csv file  
with open(FilePath, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    csvwriter.writerow(FeaturesList)  
        
    # writing the data rows  
    csvwriter.writerows(zeros_array) 


Reading TF Dataset from CSV file. Makes it lot more easier to load data than process it everytime.

In [ ]:
import pandas as pd

FilePath = '/content/drive/My Drive/Consolidated_TF_EmailCSV.csv'

TF_DataSet = pd.read_csv(FilePath)

TF_DataSet.head()

,136,heathrow,1100-1800,embryogense,40351,corey,observer,odonnaile,808-942,86d,fortunecity,tient,160-171,wiren,romajus,gamer,anhalt,pragmaphilological,savickiene,igy,310-823,junk,arnaiz,search-and,11391,bellus,salvat,especial,eckhardt,hiraus,restaurant,jay,stately,erosion,90095-1543,computer-base,elliston,lennard,verkehrssprachen,range,...,ciao,aghast,type-set,20846,732,atholic,orientalium,finnischen,maddieson,videophone,94720-2580,calvus,225,harangue,kozonoj,0855-22,gelso,iphi,11733,grants,eigenen,transducers,biterly,copies,palmada,387-427,yuha,deviate,cleanest,cracovie,base64,introducao,raleigh,msus,telo,wante,multi-tap,intellect,scarfield,SPAM
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Load Binary Dataset from CSV File.

In [48]:
import pandas as pd

FilePath = '/content/drive/My Drive/ConsolidatedEmailCSV.csv'

DataSet = pd.read_csv(FilePath)

DataSet.head()

,136,heathrow,1100-1800,embryogense,40351,corey,observer,odonnaile,808-942,86d,fortunecity,tient,160-171,wiren,romajus,gamer,anhalt,pragmaphilological,savickiene,igy,310-823,junk,arnaiz,search-and,11391,bellus,salvat,especial,eckhardt,hiraus,restaurant,jay,stately,erosion,90095-1543,computer-base,elliston,lennard,verkehrssprachen,range,...,ciao,aghast,type-set,20846,732,atholic,orientalium,finnischen,maddieson,videophone,94720-2580,calvus,225,harangue,kozonoj,0855-22,gelso,iphi,11733,grants,eigenen,transducers,biterly,copies,palmada,387-427,yuha,deviate,cleanest,cracovie,base64,introducao,raleigh,msus,telo,wante,multi-tap,intellect,scarfield,SPAM
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Information Gain Calculations are carried out Below.

*   The Information Gain is also later written to the file for easy of loading next time when I work with it. 
*   Occurance is considered not Frequency.
*   (Takes a lot of time to calculate, use already written CSV)







In [ ]:
 
import math

DataSetLen = len(DataSet["SPAM"])
print("DataSetLen",DataSetLen)
SpamArry = [0] * DataSetLen
SpamArry = [int(sub) for sub in DataSet["SPAM"]] 

count = 0

for i in SpamArry:
  if i == 1:
    count+=1
Legitp = float((DataSetLen - count)/DataSetLen)
Legit_Count = (DataSetLen - count)
print("Legit_Count ",Legit_Count)
Spam_Count = count
print("Spam_Count ", Spam_Count)
#print(Legitp)
spamp = float(count/DataSetLen)
#print(spamp)
p1  = -spamp*math.log(spamp,10)
p2  = -(1-spamp)*math.log((1-spamp),10)
H_of_C = p1 + p2
#print(p1+p2)

Present_given_spam = 0
Not_Present_given_spam = 0
Present_given_Ham = 0
Not_Present_given_Ham = 0
Present = 0
Not_Present = 0
TotalEmails = DataSetLen
Word_IG_dict = {}
#print(DataSet[""])
for i in DataSet:
  if i != "SPAM":
    for j,k in enumerate(DataSet[i]):
      if int(k) == 1:
        Present+=1
        if int(DataSet["SPAM"][j]) == 1:
          Present_given_spam += 1
          #print("Present_given_spam: ", Present_given_spam)
        else:
          Present_given_Ham += 1
          #print("Present_given_Ham: ", Present_given_Ham)
      elif int(k) == 0:
        Not_Present+=1
        if int(DataSet["SPAM"][j]) == 1:
          Not_Present_given_spam += 1
          #print("Not_Present_given_spam: ", Not_Present_given_spam)
        else:
          Not_Present_given_Ham += 1
          #print("Not_Present_given_Ham: ", Not_Present_given_Ham)
    
    
    if Not_Present!=0:
      P_Present = float(Present/TotalEmails)
      P_Not_Present = float(Not_Present/TotalEmails)

      P_Present_given_Spam = float((Present_given_spam+1)/(Spam_Count+2))
      Joint_Prob_Present_Spam = P_Present_given_Spam * spamp 
      Prob_Spam_given_Present = math.log(P_Present_given_Spam * spamp / P_Present,10)


      P_Not_Present_given_Spam = float((Not_Present_given_spam+1)/(Spam_Count+2))
      Joint_Prob_Not_Present_Spam = P_Not_Present_given_Spam * spamp 
      Prob_Spam_given_Not_Present = math.log(P_Not_Present_given_Spam * spamp / P_Not_Present,10)
            

      P_Present_given_Ham = float((Present_given_Ham+1)/(Legit_Count+2))      
      Joint_Prob_Present_Ham = P_Present_given_Ham * Legitp 
      Prob_Ham_given_Present = math.log(P_Present_given_Ham * Legitp /P_Present,10)


      P_Not_Present_given_Ham = float((Not_Present_given_Ham+1)/(Legit_Count+2))
      Joint_Prob_Not_Present_Ham = P_Not_Present_given_Ham * Legitp 
      Prob_Ham_given_Not_Present = math.log(P_Not_Present_given_Ham * Legitp/P_Not_Present,10)
          
      H_of_C_given_X = (-1.0)* (Joint_Prob_Present_Spam*Prob_Spam_given_Present + Joint_Prob_Not_Present_Spam*Prob_Spam_given_Not_Present + Joint_Prob_Present_Ham*Prob_Ham_given_Present + Joint_Prob_Not_Present_Ham*Prob_Ham_given_Not_Present)

      IG = H_of_C - H_of_C_given_X
      if str(i) not in Word_IG_dict:
        Word_IG_dict[i] = IG
    else:
      print("0 Gain feature: ", i)
    #first_term = Joint_Prob_Present_Spam * Prob_Spam_given_Present


    Present_given_spam = 0
    Not_Present_given_spam = 0
    Present_given_Ham = 0
    Not_Present_given_Ham = 0
    Present = 0
    Not_Present = 0
    #print(first_term)
    






DataSetLen 2893
Legit_Count  2412
Spam_Count  481
0 Gain feature:  Subject:


Information Gain Is written to CSV for fasters processing 
(Developer Note >> USE ONCE)

In [ ]:
import csv 
FilePathIGmap = '/content/drive/My Drive/IGmap.csv'

# writing to csv file  
with open(FilePathIGmap, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)    
        
    # writing the data rows  
    csvwriter.writerows(Word_IG_dict.items()) 

 Read IG from CSV                    (Developer Note >>>  Start Here) 

In [ ]:
import csv 
FilePathIGmap = '/content/drive/My Drive/IGmap.csv'
fetchedIG = {}

with open(FilePathIGmap) as csvfile:
    
    readCSV = csv.reader(csvfile, delimiter=',')
    
    for row in readCSV:
      fetchedIG[row[0]] = float(row[1])

#print(fetchedIG)

## Top 10 Features Printed Below
Getting Top N Features ( N = 10, 100, 1000)

In [ ]:
Sorted_IG_dict={ k:v for k,v in sorted(fetchedIG.items(), key = lambda x: x[1], reverse=True) }
# print(Sorted_IG_dict[:11])
Num = 1
FCombLen = 3
Top10features = []
Top100Features = []
Top1000Features = []
VarFeatures = [[] for i in range(FCombLen)]
#print(len(VarFeatures))
for Key in Sorted_IG_dict:
    #print( Val, " ", fetchedIG[Val])
    #TopNfeatures.append(Key) 
    if Num <= 10:
      Top10features.append(Key) 
      if Num == 10:
        VarFeatures[0] = Top10features  
    if Num <= 100:
      Top100Features.append(Key)
      if Num == 100:
        VarFeatures[1] = Top100Features
    if Num <= 1000:
      Top1000Features.append(Key)
      if Num == 1000: 
        VarFeatures[2] = Top1000Features
        break
    Num+=1

#print(VarFeatures)
# for i in VarFeatures:
#   print(len(i))

print("Top 10 Features: ")
print(Top10features)

Top 10 Features: 
['!', 'language', 'remove', 'free', 'university', 'linguistic', 'money', 'click', '@', 'our']


Calculating Feature Instances in Dataset

In [ ]:
# Part 1 to - Part 9 training 

#part 10 total files
filenum = 291

map_Feature_Given_Spam = {}
map_Feature_Given_Ham = {}
 
DataSetLen = len(DataSet["SPAM"])
print("DataSetLen",DataSetLen)
SpamArry = [0] * DataSetLen
SpamArry = [int(sub) for sub in DataSet["SPAM"]] 

count = 0

for i in SpamArry:
  if i == 1:
    count+=1

Legitp = float((DataSetLen - count)/DataSetLen)
Legit_Count = (DataSetLen - count)
print("Legit_Count ",Legit_Count)
Spam_Count = count
print("Spam_Count ", Spam_Count)
#print(Legitp)
spamp = float(count/DataSetLen)
#print(spamp)

# Selecting the highest Feature list
for k in Top1000Features:
  map_Feature_Given_Spam[k] = 0
  map_Feature_Given_Ham[k] = 0

#Map of Features for 1 to 9 parts
for i in DataSet:
  if i != "SPAM" and i in Top1000Features:
    for j in range(0,(len(DataSet[i])-filenum)):
      if int(DataSet[i][j]) == 1 and int(DataSet["SPAM"][j]) == 1:
        map_Feature_Given_Spam[i]+=1
      elif int(DataSet[i][j]) == 1 and int(DataSet["SPAM"][j]) == 0:
        map_Feature_Given_Ham[i]+=1



# Spam_Count and Legit_Count - part10 outputs 
TotalEmails = len(SpamArry)
SpamCnt1to9prt = Spam_Count
HamCnt1to9prt = Legit_Count
TotalTest = 0
for i in range(TotalEmails-filenum,TotalEmails):
  TotalTest +=1
  if SpamArry[i] == 0:
    HamCnt1to9prt -= 1
  else:
    SpamCnt1to9prt -= 1 

print("Total Spam_Count",Spam_Count)
print("Total Legit_Count",Legit_Count)
print("Total Test Files",TotalTest)
print("Spam Count in 1 to 9 parts: ",SpamCnt1to9prt)
print("Ham Count in 1 to 9 parts: ",HamCnt1to9prt)
# print("Feature Given Ham: ")
# print(map_Feature_Given_Ham)
# print("Feature Given Spam")
# print(map_Feature_Given_Spam)

DataSetLen 2893
Legit_Count  2412
Spam_Count  481
Total Spam_Count 481
Total Legit_Count 2412
Total Test Files 291
Spam Count in 1 to 9 parts:  432
Ham Count in 1 to 9 parts:  2170


Feature Given Spam and Feature Given Legit /Ham Probability Calculation

In [ ]:
map_Prob_Feature_Given_Ham = {}
map_Prob_Feature_Given_Spam = {k:float((v+1)/(SpamCnt1to9prt+2)) for k,v in map_Feature_Given_Spam.items()}
for Key,Val in map_Feature_Given_Ham.items():
  map_Prob_Feature_Given_Ham[Key] = float((Val+1)/(HamCnt1to9prt+2))

print("Probability of Feature Given Ham Map Below:")
print(map_Prob_Feature_Given_Ham)
print("Prbability of Feature Given Spam Map Below:")
print(map_Prob_Feature_Given_Spam)

Calculating Feature Instances in TF Dataset of Term Frequencies (TF)

In [ ]:
#part 10 total files
filenum = 291

map_TF_Feature_Given_Spam = {}
map_TF_Feature_Given_Ham = {}
 
# TF_DataSetLen = len(DataSetTF["SPAM"])
# print("DataSetLen",TF_DataSetLen)
# SpamArry = [0] * TF_DataSetLen
# SpamArry = [int(sub) for sub in DataSetTF["SPAM"]] 

# Selecting the highest Feature list
for k in Top1000Features:
  map_TF_Feature_Given_Spam[k] = 0
  map_TF_Feature_Given_Ham[k] = 0

#Map of Features for 1 to 9 parts
for i in TF_DataSet:
  if i != "SPAM" and i in Top1000Features:
    for j in range(0,(len(TF_DataSet[i])-filenum)):
      if int(TF_DataSet[i][j]) > 0 and int(TF_DataSet["SPAM"][j]) == 1:
        map_TF_Feature_Given_Spam[i]+= int(TF_DataSet[i][j])
      elif int(TF_DataSet[i][j]) > 0 and int(TF_DataSet["SPAM"][j]) == 0:
        map_TF_Feature_Given_Ham[i]+=int(TF_DataSet[i][j])

print("TF Feature Given Ham: ")
print(map_TF_Feature_Given_Ham)
print("TF Feature Given Spam: ")
print(map_TF_Feature_Given_Spam)

Write TF Feature Presence Count in Ham and Spam for easy retrival

In [ ]:
import csv 
FilePathFeatureGSpamTF = '/content/drive/My Drive/map_TF_Feature_Given_Spam.csv'
FilePathFeatureGHamTF = '/content/drive/My Drive/map_TF_Feature_Given_Ham.csv'

# writing to csv file  
with open(FilePathFeatureGHamTF, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)    
        
    # writing the data rows  
    csvwriter.writerows(map_TF_Feature_Given_Ham.items()) 

# writing to csv file  
with open(FilePathFeatureGSpamTF, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)    
        
    # writing the data rows  
    csvwriter.writerows(map_TF_Feature_Given_Spam.items()) 

Read TF Feature Count from CSV

In [ ]:
import csv 
fetchedIG = {}
FilePathFeatureGSpamTF = '/content/drive/My Drive/map_TF_Feature_Given_Spam.csv'
FilePathFeatureGHamTF = '/content/drive/My Drive/map_TF_Feature_Given_Ham.csv'
map_TF_Feature_Given_Ham = {}
map_TF_Feature_Given_Spam = {}
with open(FilePathFeatureGHamTF) as csvfile:
    
    readCSV = csv.reader(csvfile, delimiter=',')
    
    for row in readCSV:
      map_TF_Feature_Given_Ham[row[0]] = float(row[1])

with open(FilePathFeatureGSpamTF) as csvfile:
    
    readCSV = csv.reader(csvfile, delimiter=',')
    
    for row in readCSV:
      map_TF_Feature_Given_Spam[row[0]] = float(row[1])

Total Words in Spam and Total Words in Ham calculated below!  - (Developer Note >>> Takes a Lot of Time, run once)

In [ ]:
Total_Words_Spam = 0
Total_Words_Ham = 0
for i in TF_DataSet:
  for j in range(0,len(TF_DataSet[i])-filenum):
    if int(TF_DataSet[i][j]) > 0 and int(TF_DataSet["SPAM"][j]) == 1:
        Total_Words_Spam += int(TF_DataSet[i][j])
    elif int(TF_DataSet[i][j]) > 0 and int(TF_DataSet["SPAM"][j]) == 0:
        Total_Words_Ham+=int(TF_DataSet[i][j])
print(Total_Words_Spam)
print(Total_Words_Ham)

297052
1000879


Calculating Feature Probabilty for TF

In [ ]:
# Use for faster calculations
# Total Spam word - 250513
# Total Ham words - 785662
# size of Laplasian Denominator 
Total_Words_Spam = 297052
Total_Words_Ham = 1000879


#M = len(map_Feature_Given_Spam)
#print("M :", M)
# For M = 1000
map_TF_Prob_Feature_Given_Ham = {k:float((v+1)/(Total_Words_Ham+1000)) for k,v in map_TF_Feature_Given_Ham.items()}
map_TF_Prob_Feature_Given_Spam = {k:float((v+1)/(Total_Words_Spam+1000)) for k,v in map_TF_Feature_Given_Spam.items()}

# For M = 100
map_TF_M100_Prob_Feature_Given_Ham = {k:float((v+1)/(Total_Words_Ham+100)) for k,v in map_TF_Feature_Given_Ham.items()}
map_TF_M100_Prob_Feature_Given_Spam = {k:float((v+1)/(Total_Words_Spam+100)) for k,v in map_TF_Feature_Given_Spam.items()}

# For M = 10
map_TF_M10_Prob_Feature_Given_Ham = {k:float((v+1)/(Total_Words_Ham+10)) for k,v in map_TF_Feature_Given_Ham.items()}
map_TF_M10_Prob_Feature_Given_Spam = {k:float((v+1)/(Total_Words_Spam+10)) for k,v in map_TF_Feature_Given_Spam.items()}

#print(map_TF_Prob_Feature_Given_Ham)
#print(map_TF_Prob_Feature_Given_Spam)

Write Feature Probability so its easier to load when needed! B-NB


In [ ]:
import csv 
FilePathFeatureGHam = '/content/drive/My Drive/map_Prob_Feature_Given_Ham.csv'
FilePathFeatureGSpam = '/content/drive/My Drive/map_Prob_Feature_Given_Spam.csv'

# writing to csv file  
with open(FilePathFeatureGHam, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)    
        
    # writing the data rows  
    csvwriter.writerows(map_Prob_Feature_Given_Ham.items()) 

# writing to csv file  
with open(FilePathFeatureGSpam, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)    
        
    # writing the data rows  
    csvwriter.writerows(map_Prob_Feature_Given_Spam.items()) 

Read Feature Probability given ham/spam for Models B-NB

In [ ]:
import csv 
fetchedIG = {}
FilePathFeatureGHam = '/content/drive/My Drive/map_Prob_Feature_Given_Ham.csv'
FilePathFeatureGSpam = '/content/drive/My Drive/map_Prob_Feature_Given_Spam.csv'
map_Prob_Feature_Given_Ham = {}
map_Prob_Feature_Given_Spam = {}
with open(FilePathFeatureGHam) as csvfile:
    
    readCSV = csv.reader(csvfile, delimiter=',')
    
    for row in readCSV:
      map_Prob_Feature_Given_Ham[row[0]] = float(row[1])

with open(FilePathFeatureGSpam) as csvfile:
    
    readCSV = csv.reader(csvfile, delimiter=',')
    
    for row in readCSV:
      map_Prob_Feature_Given_Spam[row[0]] = float(row[1])

## All Classifier Precision, Recall, accuracy, confusion Matrix output.

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
Pred_Ouput = []
Spam_Pred = []
Ham_Pred = []

MN_Spam_Pred = []
MN_HamPred = []
MN_Pred_Ouput = []

Prob_Spam1to9 = float(SpamCnt1to9prt/(SpamCnt1to9prt+HamCnt1to9prt))
Prob_Ham1to9 = float(HamCnt1to9prt/(SpamCnt1to9prt+HamCnt1to9prt))

ActualOutput = [SpamArry[i] for i in range(TotalEmails-filenum, TotalEmails)]
len(ActualOutput)
for TopNfeatures in VarFeatures:
  Pred_Ouput = []
  Spam_Pred = []
  Ham_Pred = []

  MN_Spam_Pred = []
  MN_HamPred = []
  MN_Pred_Ouput = []
  
  TF_Pred_Output = []
  TF_Ham_Pred = []
  TF_Spam_Pred = []
  for j in range(TotalEmails-filenum, TotalEmails):
    FeatureBits = {}
    for i in DataSet:
      if i !="SPAM" and i in TopNfeatures:
        if int(DataSet[i][j]) == 1:
          FeatureBits[i] = 1   
        else:
          FeatureBits[i] = 0
    #print(FeatureBits)
    SpamPredProb_var = 1.0*Prob_Spam1to9
    HamPredProb_var = 1.0*Prob_Ham1to9

    MN_SpamPredProb_var = 1.0*Prob_Spam1to9
    MN_HamPredProb_var = 1.0*Prob_Ham1to9

    TF_SpamPredProb_var = 1.0*Prob_Spam1to9
    TF_HamPredProb_var = 1.0*Prob_Ham1to9
    
    for K,V in FeatureBits.items():
      if V == 1:
        SpamPredProb_var *= map_Prob_Feature_Given_Spam[K]
        HamPredProb_var *= map_Prob_Feature_Given_Ham[K]
        MN_SpamPredProb_var *= map_Prob_Feature_Given_Spam[K]
        MN_HamPredProb_var *= map_Prob_Feature_Given_Ham[K]

        if len(TopNfeatures) == 1000:
           TF_SpamPredProb_var *= map_TF_Prob_Feature_Given_Spam[K]**TF_DataSet[K][j]
           TF_HamPredProb_var *= map_TF_Prob_Feature_Given_Ham[K]**TF_DataSet[K][j]
        if len(TopNfeatures) == 100:
           TF_SpamPredProb_var *= map_TF_M100_Prob_Feature_Given_Spam[K]**TF_DataSet[K][j]
           TF_HamPredProb_var *= map_TF_M100_Prob_Feature_Given_Ham[K]**TF_DataSet[K][j]
        if len(TopNfeatures) == 10:
           TF_SpamPredProb_var *= map_TF_M10_Prob_Feature_Given_Spam[K]**TF_DataSet[K][j]
           TF_HamPredProb_var *= map_TF_M10_Prob_Feature_Given_Ham[K]**TF_DataSet[K][j]
      else:
        #print(K, V)
        SpamPredProb_var *= (1.0 - map_Prob_Feature_Given_Spam[K])
        HamPredProb_var *= (1.0 - map_Prob_Feature_Given_Ham[K])

    Spam_Pred.append(SpamPredProb_var)
    Ham_Pred.append(HamPredProb_var)

    MN_Spam_Pred.append(MN_SpamPredProb_var)
    MN_HamPred.append(MN_HamPredProb_var)

    TF_Ham_Pred.append(TF_HamPredProb_var)
    TF_Spam_Pred.append(TF_SpamPredProb_var)

    if SpamPredProb_var>=HamPredProb_var:
      Pred_Ouput.append(1)
      #print(i)
    else:
      Pred_Ouput.append(0)  

    if MN_SpamPredProb_var>= MN_HamPredProb_var:
      MN_Pred_Ouput.append(1)
    else:
      MN_Pred_Ouput.append(0)
    
    if TF_SpamPredProb_var>= TF_HamPredProb_var:
      TF_Pred_Output.append(1)
    else:
      TF_Pred_Output.append(0)

    #print(len(Pred_Ouput))

  CorrectPred = 0
  for i in range(filenum):
    if Pred_Ouput[i] ==  ActualOutput[i]:
      CorrectPred+=1

    # for i,j in zip(MN_Pred_Ouput, Pred_Ouput):
    #   print(i,j)

  MN_CorrectPred = 0
  for i in range(filenum):
    if MN_Pred_Ouput[i] ==  ActualOutput[i]:
      MN_CorrectPred+=1
   
  TF_CorrectPred = 0
  for i in range(filenum):
    if TF_Pred_Output[i] ==  ActualOutput[i]:
      TF_CorrectPred+=1

  #print(Spam_Pred)
  
  print(f"---------Feature Length {len(TopNfeatures)} ----------")
  print("\n---------Accuracy Score---------------")
  print(f"Feature Length {len(TopNfeatures)} Accuracy Bernoulli NB: ", float(CorrectPred/filenum)*100.0)
  print(f"Feature Length {len(TopNfeatures)} Accuracy Multinomial NB: ", float(MN_CorrectPred/filenum)*100.0)
  print(f"Feature Length {len(TopNfeatures)} Accuracy Multinomial NB TF: ", float(TF_CorrectPred/filenum)*100.0)
  print("\n---------Precision Score--------------")
  print(f"Feature Length {len(TopNfeatures)} Precision Bernoulli NB: ", precision_score(ActualOutput, Pred_Ouput))
  print(f"Feature Length {len(TopNfeatures)} Precision Multinomial NB: ", precision_score(ActualOutput, MN_Pred_Ouput))
  print(f"Feature Length {len(TopNfeatures)} Precision Multinomial NB TF: ", precision_score(ActualOutput, TF_Pred_Output))
  print("\n---------Recall Score-----------------")
  print(f"Feature Length {len(TopNfeatures)} Recall Bernoulli NB: ", recall_score(ActualOutput, Pred_Ouput))
  print(f"Feature Length {len(TopNfeatures)} Recall Multinomial NB: ", recall_score(ActualOutput, MN_Pred_Ouput))
  print(f"Feature Length {len(TopNfeatures)} Recall Multinomial NB TF: ", recall_score(ActualOutput, TF_Pred_Output))
  print("\n---------Confusion Matrix-------------")
  print("Bernoulli NB: ")
  print(confusion_matrix(ActualOutput, Pred_Ouput))
  print("Multinomial NB: ")
  print(confusion_matrix(ActualOutput, MN_Pred_Ouput))
  print("Multinomial NB TF: ")
  print(confusion_matrix(ActualOutput, TF_Pred_Output))
  print("\n-------Classification Report----------")
  print("Bernoulli NB: ")
  print(classification_report(ActualOutput, Pred_Ouput))
  print("Multinomial NB: ")
  print(classification_report(ActualOutput, MN_Pred_Ouput))
  print("Multinomial NB TF: ")
  print(classification_report(ActualOutput, TF_Pred_Output)) 
  print("#######################################################")
    


---------Feature Length 10 ----------

---------Accuracy Score---------------
Feature Length 10 Accuracy Bernoulli NB:  96.21993127147766
Feature Length 10 Accuracy Multinomial NB:  96.21993127147766
Feature Length 10 Accuracy Multinomial NB TF:  97.2508591065292

---------Precision Score--------------
Feature Length 10 Precision Bernoulli NB:  0.9318181818181818
Feature Length 10 Precision Multinomial NB:  0.9318181818181818
Feature Length 10 Precision Multinomial NB TF:  0.9555555555555556

---------Recall Score-----------------
Feature Length 10 Recall Bernoulli NB:  0.8367346938775511
Feature Length 10 Recall Multinomial NB:  0.8367346938775511
Feature Length 10 Recall Multinomial NB TF:  0.8775510204081632

---------Confusion Matrix-------------
Bernoulli NB: 
[[239   3]
 [  8  41]]
Multinomial NB: 
[[239   3]
 [  8  41]]
Multinomial NB TF: 
[[240   2]
 [  6  43]]

-------Classification Report----------
Bernoulli NB: 
              precision    recall  f1-score   support

        

## Attacker's Strategy; False Negative Rate Before and After Attacker's Strategy Modified test emails 

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from math import log
Spam_Pred = []
Ham_Pred = []
Pred_Output = []
AS_Spam_Pred = []
AS_Ham_Pred = []
AS_Pred_Output = []
ActualOutput = [SpamArry[i] for i in range(TotalEmails-filenum, TotalEmails)]
modifications = [0 for i in range(filenum)]
Total_SpamsModified = 0
Total_modifications = 0
FeaGvnHam = {}
FeaGvnSpam = {}
LO_of_Feature = {}
Ctr = 0
for K,V in map_Prob_Feature_Given_Ham.items():
  if Ctr <10:
    FeaGvnHam[K] = V
  Ctr+=1
Ctr = 0
for K,V in map_Prob_Feature_Given_Spam.items():
  if Ctr <10:
    FeaGvnSpam[K] = V
  Ctr+=1


for Feature in Top10features:
  LO_of_Feature[Feature] = log(float(FeaGvnSpam[Feature]/FeaGvnHam[Feature]),10)
#print(LO_of_Feature)

sorted_LO_of_Feature =  {k:v for k,v in sorted(LO_of_Feature.items(), key=lambda kv: kv[1])}

#print(sorted_FeaGvnHam)
Modification_Index = 0
Test_DataSet_Index_Ctr = 0
for j in range(TotalEmails-filenum, TotalEmails):
    FeatureBits = {}
    LO_GvntestEmail = 0
    for i in Top10features:
      if int(DataSet[i][j]) == 1:
        FeatureBits[i] = 1   
      else:
        FeatureBits[i] = 0
    original_length = 0
    for K,V in FeatureBits.items():
      if V == 1:
        LO_GvntestEmail += LO_of_Feature[K]
        original_length+=1
    
    new_Feature_length = original_length
    ModCount = 0
    AdaptModified = False
    AS_LO_GvntestEmail = LO_GvntestEmail
    #AS_HamPredProb_var = HamPredProb_var
    if AS_LO_GvntestEmail > 0:
      Pred_Output.append(1)
      # in order to only apply attacker's strategy to spam Emails, this condition checks the same.
      if ActualOutput[Test_DataSet_Index_Ctr] == 1:
        for K,Val in sorted_LO_of_Feature.items():
          if FeatureBits[K] == 0:
            AS_LO_GvntestEmail += sorted_LO_of_Feature[K]
            ModCount+=1
            new_Feature_length += 1
          if AS_LO_GvntestEmail < 0:
            modifications[Modification_Index] = ModCount
            Total_SpamsModified+=1
            Total_modifications+=ModCount
            #print("Original Feature Length: ", original_length, "New Feature Length: ", new_Feature_length)
            AdaptModified = True
            break
    else:
      Pred_Output.append(0)

    Modification_Index+=1
    Test_DataSet_Index_Ctr+=1

    if AdaptModified:
      AS_Pred_Output.append(0)
    else:
      AS_Pred_Output.append(Pred_Output[len(Pred_Output)-1])


print("Confusion Matrix of Vanilla Multinomial NB: ")
tn, fp, fn, tp = confusion_matrix(ActualOutput, Pred_Output).ravel()
#print("False Negatives: ", fn)
#print("False positive: ", fp)
#print("True Negatives: ", tn)
FNR = fn/(tp+fn)

print(confusion_matrix(ActualOutput, Pred_Output))
print("----------------------------")
print("False Negative Rate: ", FNR)
print("----------------------------")
print(classification_report(ActualOutput, Pred_Output))
print("Confusion Matrix after applying Attackers Function on test emails; Output: ")
tn, fp, fn, tp = confusion_matrix(ActualOutput, AS_Pred_Output).ravel()
FNR = fn/(tp+fn)
print(confusion_matrix(ActualOutput, AS_Pred_Output))
print("----------------------------")
print("False Negative Rate: ", FNR)
print("----------------------------")
print("----------------------------")
print("Cost: ", float(Total_modifications/Total_SpamsModified))
print("----------------------------")
print(classification_report(ActualOutput, AS_Pred_Output))
#print(modifications)


Confusion Matrix of Vanilla Multinomial NB: 
[[236   6]
 [  3  46]]
----------------------------
False Negative Rate:  0.061224489795918366
----------------------------
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       242
           1       0.88      0.94      0.91        49

    accuracy                           0.97       291
   macro avg       0.94      0.96      0.95       291
weighted avg       0.97      0.97      0.97       291

Confusion Matrix after applying Attackers Function on test emails; Output: 
[[236   6]
 [ 49   0]]
----------------------------
False Negative Rate:  1.0
----------------------------
----------------------------
Cost:  1.7391304347826086
----------------------------
              precision    recall  f1-score   support

           0       0.83      0.98      0.90       242
           1       0.00      0.00      0.00        49

    accuracy                           0.81       291
   macro avg       



##   Defender's Strategy
Passing the Attackers function/Strategy to Classifier - In other words Train Classifier on Traning Dataset altered/modified using Attackers Function/Strategy

In [ ]:
Total_modifications = 0
Total_SpamsModified = 0
Zpy_DataSet = DataSet.copy(deep=True)
Modification_Index = 0
filenum = 291
modifications = [0 for i in range(len(Zpy_DataSet["SPAM"]))]
for j in range(0,(len(Zpy_DataSet["SPAM"])-filenum)):
    FeatureBits = {}
    LO_of_trainEmail = 0
    for i in Top10features:
      if int(Zpy_DataSet[i][j]) == 1:
        FeatureBits[i] = 1   
      else:
        FeatureBits[i] = 0
    
    #print('-----------------------------------------')
    for K,V in FeatureBits.items():
        if V == 1:
          Zpy_DataSet[K][j] == 1
          #print(K) 
    original_length = 0
    for K,V in FeatureBits.items():
      if V == 1:
        LO_of_trainEmail += LO_of_Feature[K]
        original_length += 1

    AS_LO_of_trainEmail = LO_of_trainEmail
    Mod_count = 0
    AdaptModified = False
    new_Feature_length = original_length
    if AS_LO_of_trainEmail > 0: 
      # in order to only apply attacker's strategy to spam Emails, this condition checks the same.
      if Zpy_DataSet["SPAM"][j] == 1:
        for K,Val in sorted_LO_of_Feature.items():
          if FeatureBits[K] == 0:
            AS_LO_of_trainEmail += sorted_LO_of_Feature[K]
            Mod_count+=1
            FeatureBits[K] = 1
            new_Feature_length += 1
          if AS_LO_of_trainEmail < 0:
            modifications[Modification_Index] = Mod_count
            Total_SpamsModified+=1
            Total_modifications+=Mod_count
            #print("Original Feature Length: ", original_length, "New Feature Length: ", new_Feature_length)
            AdaptModified = True
            break    
  
    Modification_Index+=1
    if AdaptModified:
      for K,V in FeatureBits.items():
        if V == 1:
          Zpy_DataSet[K][j] = 1
          #print(K)     
    #print('-----------------------------------------')
    
# if Total_SpamsModified != 0:
#    print(float(Total_modifications/Total_SpamsModified))

print("Total Spam Mails Modified: ", Total_SpamsModified)

Total Spam Mails Modified:  397


Calculating New values of Feature Instances after Defender's Strategy 

In [ ]:
# Part 1 to - Part 9 training 

#part 10 total files
filenum = 291

map_Feature_Given_Spam = {}
map_Feature_Given_Ham = {}
 
DataSetLen = len(Zpy_DataSet["SPAM"])
#print("DataSetLen",DataSetLen)
SpamArry = [0] * DataSetLen
SpamArry = [int(sub) for sub in Zpy_DataSet["SPAM"]] 

count = 0

for i in SpamArry:
  if i == 1:
    count+=1

Legitp = float((DataSetLen - count)/DataSetLen)
Legit_Count = (DataSetLen - count)
#print("Legit_Count ",Legit_Count)
Spam_Count = count
#print("Spam_Count ", Spam_Count)
#print(Legitp)
spamp = float(count/DataSetLen)
#print(spamp)

# Selecting the highest Feature list
for k in Top10features:
  map_Feature_Given_Spam[k] = 0
  map_Feature_Given_Ham[k] = 0

#Map of Features for 1 to 9 parts
for i in Zpy_DataSet:
  if i in Top10features:
    for j in range(0,(len(Zpy_DataSet[i])-filenum)):
      if int(Zpy_DataSet[i][j]) == 1 and int(Zpy_DataSet["SPAM"][j]) == 1:
        map_Feature_Given_Spam[i]+=1
      elif int(Zpy_DataSet[i][j]) == 1 and int(Zpy_DataSet["SPAM"][j]) == 0:
        map_Feature_Given_Ham[i]+=1

print(map_Feature_Given_Ham)
print(map_Feature_Given_Spam)

# Spam_Count and Legit_Count - part10 outputs 
TotalEmails = len(SpamArry)
SpamCnt1to9prt = Spam_Count
HamCnt1to9prt = Legit_Count
TotalTest = 0
for i in range(TotalEmails-filenum,TotalEmails):
  TotalTest +=1
  if SpamArry[i] == 0:
    HamCnt1to9prt -= 1
  else:
    SpamCnt1to9prt -= 1 

print(" Spam_Count",Spam_Count)
print(" Legit_Count",Legit_Count)
#print(" TotalTest",TotalTest)
print(" Spam Count 1 to 9 part",SpamCnt1to9prt)
print(" Ham Count 1 to 9 part",HamCnt1to9prt)

{'!': 337, 'language': 1447, 'remove': 22, 'free': 129, 'university': 1276, 'linguistic': 1105, 'money': 58, 'click': 13, '@': 1564, 'our': 367}
{'!': 364, 'language': 245, 'remove': 185, 'free': 251, 'university': 116, 'linguistic': 397, 'money': 169, 'click': 116, '@': 116, 'our': 260}
 Spam_Count 481
 Legit_Count 2412
 Spam Count 1 to 9 part 432
 Ham Count 1 to 9 part 2170


Calculating New weights/ Feature Probabilities after applying Defender's Strategy

In [ ]:
map_Prob_Feature_Given_Ham = {}
map_Prob_Feature_Given_Spam = {k:float((v+1)/(SpamCnt1to9prt+2)) for k,v in map_Feature_Given_Spam.items()}
for Key,Val in map_Feature_Given_Ham.items():
  map_Prob_Feature_Given_Ham[Key] = float((Val+1)/(HamCnt1to9prt+2))

#print(map_Prob_Feature_Given_Ham)
#print(map_Prob_Feature_Given_Spam)

    

## Applying Defender's strategy trained Classifier on (Modified through attacker's strategy) test DataSet and checking the False Positive and False Negative Rate

*   While the Classifier is adept at classifying Spam Mails as Spam - which is suggested by Lower False Negative Rate, after tuning training data with Defender's strategy.

*   While it drastically improves on False Negativity, but it deteriorates its ability to categories Ham mails as Ham effectively. This is evident in the False Positives recorded.  

*   Due to the above the accuracy of the model is hampered, but as a result of this activity the model is more robust and less overfitting.





In [ ]:

len(ActualOutput)

Pred_Ouput = []
Spam_Pred = []
Ham_Pred = []

MN_Spam_Pred = []
MN_HamPred = []
MN_Pred_Ouput = []

TF_Pred_Output = []
TF_Ham_Pred = []
TF_Spam_Pred = []
for j in range(TotalEmails-filenum, TotalEmails):
  FeatureBits = {}
  for i in Top10features:
      if int(Zpy_DataSet[i][j]) == 1:
        FeatureBits[i] = 1   
      else:
        FeatureBits[i] = 0

  SpamPredProb_var = 1.0*Prob_Spam1to9
  HamPredProb_var = 1.0*Prob_Ham1to9
  
  for K,V in FeatureBits.items():
    if V == 1:
      SpamPredProb_var *= map_Prob_Feature_Given_Spam[K]
      HamPredProb_var *= map_Prob_Feature_Given_Ham[K]
    
  if SpamPredProb_var>=HamPredProb_var:
    Pred_Ouput.append(1)
    #print(i)
  else:
    Pred_Ouput.append(0)

print("Multinomial NB after Defender's Strategy: ")
tn, fp, fn, tp = confusion_matrix(ActualOutput, Pred_Ouput).ravel()
FNR = fn/(tp+fn)
print("Confusion Matrix: ")
print(confusion_matrix(ActualOutput, Pred_Output))
print("----------------------------")
print("False Negative Rate: ", FNR)
print("----------------------------")
FPR = fp/(tn+fp)
print("----------------------------")
print("False Positive Rate: ", FPR)
print("----------------------------")
print("----------------------------")
print("Accuracy Score: ", (tn+tp)/(filenum) * 100)
print("----------------------------")
print(classification_report(ActualOutput,Pred_Ouput))
#print(Class)


Multinomial NB after Defender's Strategy: 
Confusion Matrix: 
[[236   6]
 [  3  46]]
----------------------------
False Negative Rate:  0.16326530612244897
----------------------------
----------------------------
False Positive Rate:  0.13636363636363635
----------------------------
----------------------------
Accuracy Score:  85.91065292096219
----------------------------
              precision    recall  f1-score   support

           0       0.96      0.86      0.91       242
           1       0.55      0.84      0.67        49

    accuracy                           0.86       291
   macro avg       0.76      0.85      0.79       291
weighted avg       0.89      0.86      0.87       291



In the Above Output one can see that the flase Negative Rate has fallen considerably indicating that defender's startegy is working well.

## SVM Model Based Spam Filtering


1.   Used Grid Search CV and Random Search CV for parameters tuning. 
2.   Grid Search CV's accuracy is marginally better than Randomised Search CV's
3.   I am using the SVC() function provided by sklearn library
4.   I am using the parameters like Kernel, C, Coef0 and Gamma for training the SVM model. Beacause these are the most important parameters that affect the accuracy by considerable amount. If we select an inappropriate kernel the accuracy is hampered by a huge margin.




Training and validation data Creation

In [49]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

New_DataSet = DataSet.drop([i for i in range(len(DataSet["SPAM"])-291, len(DataSet))], axis=0)
X_Train = New_DataSet[Top10features]
Y_Train = New_DataSet["SPAM"]
#print(Y_Train)
#print(X_Train)
New_DataSet_2 = DataSet.drop([i for i in range(0, len(DataSet["SPAM"])-291)], axis=0)
#print(New_DataSet_2)
X_test = New_DataSet_2[Top10features]
Y_test = New_DataSet_2["SPAM"]
#print(X_test)
#print(Y_test)

Finding the best of Parameters from ***Kernel, C, Coef0, Degree and Gamma*** using Grid Search CV



In [50]:
parameters = {'kernel':('linear', 'poly', 'rbf','sigmoid'), 
              'C':[1,50,5],
              'degree':[3,8],
              'coef0':[0.001,10,0.5],
              'gamma':('auto','scale')}

SVModel = svm.SVC()
GridSrch = GridSearchCV(SVModel, parameters, cv = 3)
GridSrch.fit(X_Train,Y_Train)
GridSrch.best_params_

{'C': 5, 'coef0': 0.001, 'degree': 3, 'gamma': 'auto', 'kernel': 'rbf'}

Getting ***Accuracy*** of Model based on Grid Search CV On validation/Test Data

In [55]:
# Using the best parameter values from above

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
SVModel = svm.SVC( kernel='rbf', C=5, degree=3, gamma = 'auto', coef0 = 0.001)
SVModel.fit(X_Train,Y_Train)
print("Accuracy: ")
print(accuracy_score(Y_test, SVModel.predict(X_test)))
print("-------------------------------")
print("Precision SVM : ", precision_score(ActualOutput, SVModel.predict(X_test)))
print("-------------------------------")
print("Recall SVM: ", precision_score(ActualOutput, SVModel.predict(X_test)))
print("-------------------------------")  

Accuracy: 
0.9656357388316151
-------------------------------
Precision SVM :  0.9534883720930233
-------------------------------
Recall SVM:  0.9534883720930233
-------------------------------


Finding the best of Parameters from ***Kernel, C, Coef0, Degree and Gamma*** using Randomized Search CV

In [52]:
from sklearn.model_selection import RandomizedSearchCV
RandS = RandomizedSearchCV(SVModel, parameters, cv = 3)
RandS.fit(X_Train, Y_Train)
RandS.best_params_

{'C': 50, 'coef0': 0.001, 'degree': 3, 'gamma': 'auto', 'kernel': 'rbf'}

Getting **Accuracy** of Model based on Randomized Search CV parameters

In [56]:
from sklearn.metrics import accuracy_score
SVModel = svm.SVC( kernel='rbf', C=50, degree=3, gamma = 'auto', coef0 = 0.001)
SVModel.fit(X_Train,Y_Train)
print("Accuracy: ")
print(accuracy_score(Y_test, SVModel.predict(X_test)))
print("-------------------------------")
print("Precision SVM : ", precision_score(ActualOutput, SVModel.predict(X_test)))
print("-------------------------------")
print("Recall SVM: ", precision_score(ActualOutput, SVModel.predict(X_test)))
print("-------------------------------")  

Accuracy: 
0.9621993127147767
-------------------------------
Precision SVM :  0.9523809523809523
-------------------------------
Recall SVM:  0.9523809523809523
-------------------------------
